In [3]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore') 

df = pd.read_csv('adult.csv')

df.income.value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

In [5]:
## ohe_logres.py -> 로지스틱 회귀 모델

import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    
    # 폴드 값이 있는 학습 데이터를 불러온다.
    df = pd.read_csv("adult_folds.csv")
    
    # 수치형 열의 목록
    num_cols = [
    "fnlwgt",
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week"
     ]
    
    # 수치형 목록을 제거한다.
    df = df.drop(num_cols, axis=1)
    
    # 타겟 변수를 0 과 1 로 매핑한다.
    target_mapping = {
    "<=50K": 0,
    ">50K": 1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    
    # income 과 kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
    f for f in df.columns if f not in ("kfold", "income")
    ]
    
    # 모든 NaN 값을 NONE 으로 채운다
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")

    # 폴드 값으로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # 폴드 값으로 검증 데이터를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    # scikit-learn 의 OneHotEncoder 객체를 초기화 한다.
    ohe = preprocessing.OneHotEncoder()

    # 학습 + 검증 데이터로 학습한다.
    full_data = pd.concat(
    [df_train[features], df_valid[features]],
    axis=0
    )
    ohe.fit(full_data[features])

    # 학습 데이터를 변환한다.
    x_train = ohe.transform(df_train[features])

    # 검증 데이터를 변환한다.
    x_valid = ohe.transform(df_valid[features])

    # 로지스틱 회귀 모델을 초기화 한다.
    model = linear_model.LogisticRegression()

    # 변환한 데이터로 모델을 학습한다.
    model.fit(x_train, df_train.income.values)
    
    # 검증 데이터로 예측한다.
    # AUC 를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1 의 확률을 사용한다
    valid_preds = model.predict_proba(x_valid)[:, 1]
    
    # auc 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    
    # auc 값을 출력한다.
    print(f"Fold = {fold}, AUC = {auc}")
    
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)

Fold = 0, AUC = 0.8794973428337469
Fold = 1, AUC = 0.887601339079321
Fold = 2, AUC = 0.8852609687685753
Fold = 3, AUC = 0.8681264602321512
Fold = 4, AUC = 0.8728581541840037


In [10]:
## lbl_xgb.py -> xgboost

import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    
    # 폴드 값이 있는 학습 데이터를 불러온다.
    df = pd.read_csv("adult_folds.csv")
    
    # 수치형 열의 목록
    num_cols = [
    "fnlwgt",
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week"
     ]
    
    # 수치형 목록을 제거한다.
    df = df.drop(num_cols, axis=1)
    
    # 타겟 변수를 0 과 1 로 매핑한다.
    target_mapping = {
    "<=50K": 0,
    ">50K": 1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    
    # income 과 kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
    f for f in df.columns if f not in ("kfold", "income")
    ]
    
    # 모든 NaN 값을 NONE 으로 채운다
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")
        
    # 피쳐들을 레이블 인코딩한다.
    for col in features:
        
        # 각 피쳐 열에 대해 LabelEncoder를 초기화 한다.
        lbI = preprocessing.LabelEncoder()
        
        # 인코더를 모든 데이터로 학습한다.
        lbI.fit(df[col])
        
        # 모든 데이터를 변환한다.
        df.loc[:, col] = lbI.transform(df[col])

    # 폴드 값으로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # 폴드 값으로 검증 데이터를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    # 학습 데이터를 얻는다.
    x_train = df_train[features].values
    
    # 검증 데이터를 얻는다.
    x_valid = df_valid[features].values
    
    # xgb 모델을 초기화 한다.
    model = xgb.XGBClassifier(
    n_jobs = -1
    )
    
    # 변환한 데이터로 모델을 학습한다.
    model.fit(x_train, df_train.income.values)
    
    # 검증 데이터로 예측한다.
    # AUC 를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1 의 확률을 사용한다
    valid_preds = model.predict_proba(x_valid)[:, 1]
    
    # auc 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    
    # auc 값을 출력한다.
    print(f"Fold = {fold}, AUC = {auc}")
    
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)

[15:32:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 0, AUC = 0.8752407460691677
[15:32:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 1, AUC = 0.8842809878805891
[15:32:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [13]:
## lbl_xgb_num.py -> 앞에서 제거했던 수치형 변수들을 XGBoost에 추가

import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    
    # 폴드 값이 있는 학습 데이터를 불러온다.
    df = pd.read_csv("adult_folds.csv")
    
    # 수치형 열의 목록
    num_cols = [
    "fnlwgt",
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week"
     ]
    
    # 타겟 변수를 0과 1로 매핑한다.
    target_mapping = {
    }
    
    # 타겟 변수를 0 과 1 로 매핑한다.
    target_mapping = {
    "<=50K": 0,
    ">50K": 1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    
    # income 과 kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
    f for f in df.columns if f not in ("kfold", "income")
    ]
    
    # 모든 NaN 값을 NONE 으로 채운다
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        # 수치형 열은 인코딩하지 않는다.
        if col not in num_cols:
            df.loc[:, col] = df[col].astype(str).fillna('NONE')
            
    # 피쳐들을 레이블인코딩한다.
    for col in features:
        if col not in num_cols:
            # 각 피쳐 열에 대해 LabelEncoder를 초기화 한다.
            lbI = preprocessing.LabelEncoder()
        
            # 인코더를 모든 데이터로 학습한다.
            lbI.fit(df[col])
        
            # 모든 데이터를 변환한다.
            df.loc[:, col] = lbI.transform(df[col])
        
    # 폴드 값으로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # 폴드 값으로 검증 데이터를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    # 학습 데이터를 얻는다.
    x_train = df_train[features].values
    
    # 검증 데이터를 얻는다.
    x_valid = df_valid[features].values
    
    # xgb 모델을 초기화 한다.
    model = xgb.XGBClassifier(
    n_jobs = -1
    )
    
    # 변환한 데이터로 모델을 학습한다.
    model.fit(x_train, df_train.income.values)
    
    # 검증 데이터로 예측한다.
    # AUC 를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1 의 확률을 사용한다
    valid_preds = model.predict_proba(x_valid)[:, 1]
    
    # auc 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    
    # auc 값을 출력한다.
    print(f"Fold = {fold}, AUC = {auc}")
    
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)

[15:34:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 0, AUC = 0.9275682416809503
[15:34:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 1, AUC = 0.9297670258940955
[15:34:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [14]:
## lbl_xgb_num_feat.py -> 범주형 변수를 가지고 2차원 조합 구하기

import itertools
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing


def feature_engineering(df, cat_cols):
    """
    피쳐 가공을 위한 함수
    :param df: 학습/시험 데이터가 있는 pandas 데이터프레임
    :param cat_cols: 범주형 변수의 목록
    :return: 새 피쳐를 포함한 데이터프레임
    """
    # 아래 코드는 목록의 값들의 2 차원 조합을 생성한다.
    # 예를 들어:
    # list(itertools.combinations([1,2,3], 2)) ->
    # [(1, 2), (1, 3), (2, 3)]
    combi = list(itertools.combinations(cat_cols, 2))
    for c1, c2 in combi:
        df.loc[
        :,
        c1 + "_" + c2
        ] = df[c1].astype(str) + "_" + df[c2].astype(str)
    return df


def run(fold):
    
    # 폴드 값이 있는 학습 데이터를 불러온다.
    df = pd.read_csv("adult_folds.csv")
    
    # 수치형 열의 목록
    num_cols = [
    "fnlwgt",
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week"
     ]
    
    # 타겟 변수를 0과 1로 매핑한다.
    target_mapping = {
    }
    
    # 타겟 변수를 0 과 1 로 매핑한다.
    target_mapping = {
    "<=50K": 0,
    ">50K": 1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    
    # 피쳐 가공을 위한 범주형 변수의 목록
    cat_cols = [
    c for c in df.columns if c not in num_cols
    and c not in ("kfold", "income")
    ]
    
    # 새 피쳐를 추가한다.
    df = feature_engineering(df, cat_cols)
    
    # kfold와 income 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
        f for f in df.columns if f not in ('kfold', 'income')
    ]
    
    # 모든 NaN 값을 NONE 으로 채운다
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        # 수치형 열은 인코딩하지 않는다.
        if col not in num_cols:
            df.loc[:, col] = df[col].astype(str).fillna('NONE')
            
    # 피쳐들을 레이블인코딩한다.
    for col in features:
        if col not in num_cols:
            # 각 피쳐 열에 대해 LabelEncoder를 초기화 한다.
            lbI = preprocessing.LabelEncoder()
        
            # 인코더를 모든 데이터로 학습한다.
            lbI.fit(df[col])
        
            # 모든 데이터를 변환한다.
            df.loc[:, col] = lbI.transform(df[col])
            
    
    # 폴드 값으로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # 폴드 값으로 검증 데이터를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    # 학습 데이터를 얻는다.
    x_train = df_train[features].values
    
    # 검증 데이터를 얻는다.
    x_valid = df_valid[features].values
    
    # xgb 모델을 초기화 한다.
    model = xgb.XGBClassifier(
    n_jobs = -1
    )
    
    # 변환한 데이터로 모델을 학습한다.
    model.fit(x_train, df_train.income.values)
    
    # 검증 데이터로 예측한다.
    # AUC 를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1 의 확률을 사용한다
    valid_preds = model.predict_proba(x_valid)[:, 1]
    
    # auc 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    
    # auc 값을 출력한다.
    print(f"Fold = {fold}, AUC = {auc}")

    
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)

[15:36:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 0, AUC = 0.9281676897246613
[15:36:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 1, AUC = 0.9304998508808865
[15:36:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
## target_encoding.py -> 타겟 인코딩

import copy
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
import xgboost as xgb

def mean_target_encoding(data):
    
    # 데이터프레임을 복사한다.
    df = copy.deepcopy(data)
    
    # 수치형 변수의 목록
    num_cols = [
    "fnlwgt",
    "age",
    "capital.gain",
    "capital.loss",
    "hours.per.week"
    ]
    
    # 타겟 변수를 0 과 1 로 매핑한다.
    target_mapping = {
    "<=50K": 0,
    ">50K": 1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    
    # income 과 kfold 를 제외한 모든 열들을 피쳐로 사용한다.
    features = [
    f for f in df.columns if f not in ("kfold", "income")
    and f not in num_cols
    ]
    
    # 모든 NaN 값을 NONE 으로 채운다
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        # 수치형 열은 인코딩하지 않는다.
        if col not in num_cols:
            df.loc[:, col] = df[col].astype(str).fillna('NONE')
            
    # 피쳐들을 레이블인코딩한다.
    for col in features:
        if col not in num_cols:
            # 각 피쳐 열에 대해 LabelEncoder를 초기화 한다.
            lbI = preprocessing.LabelEncoder()
        
            # 인코더를 모든 데이터로 학습한다.
            lbI.fit(df[col])
        
            # 모든 데이터를 변환한다.
            df.loc[:, col] = lbI.transform(df[col])
            
    # 5 개의 검증 데이터프레임을 저장할 빈 목록
    encoded_dfs = []
    
    # 모든 폴드에 대해 계산한다.
    for fold in range(5):
    
        # 학습 데이터와 검증 데이터를 추출한다.
        df_train = df[df.kfold != fold].reset_index(drop=True)
        df_valid = df[df.kfold == fold].reset_index(drop=True)
    
        # for all feature columns, i.e. categorical columns
        for column in features:
            
            # 범주:타겟 평균의 사전을 만든다.
            mapping_dict = dict(
            df_train.groupby(column)["income"].mean()
            )
                
            # column_enc 이 평균 인코딩을 저장하는 새 열이다.
            df_valid.loc[
                :, column + "_enc"
            ] = df_valid[column].map(mapping_dict)

        # 인코딩된 검증 데이터프레임의 목록에 추가한다.
        encoded_dfs.append(df_valid)

    # 전체 데이터프레임을 생성하여 반환한다.
    encoded_df = pd.concat(encoded_dfs, axis=0)
    return encoded_df


def run(df, fold):
    # 이전과 동일한 범주를 사용한다.
    # 폴드로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop = True)
    
    # 폴드로 검증 데이러를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop = True)
    
    # income과 kfold를 제외한 모든 열들을 피쳐로 사용한다.
    features = [
        f for f in df.columns if f not in ('kfold', 'income')
    ]
    
        # 학습 데이터를 얻는다.
    x_train = df_train[features].values
    
    # 검증 데이터를 얻는다.
    x_valid = df_valid[features].values
    
    # xgb 모델을 초기화 한다.
    model = xgb.XGBClassifier(
    n_jobs = -1
    )
    
    # 변환한 데이터로 모델을 학습한다.
    model.fit(x_train, df_train.income.values)
    
    # 검증 데이터로 예측한다.
    # AUC 를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1 의 확률을 사용한다
    valid_preds = model.predict_proba(x_valid)[:, 1]
    
    # auc 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.income.values, valid_preds)
    
    # auc 값을 출력한다.
    print(f"Fold = {fold}, AUC = {auc}")

    
if __name__ == "__main__":
    # 데이터를 불러 온다.
    df = pd.read_csv('adult_folds.csv')
    
    # 타겟 인코딩을 한다.
    df = mean_target_encoding(df)
    
    # 5개의 폴드에 대해 학습과 검증을 수행한다.
    for fold_ in range(5):
        run(df, fold_)

[15:37:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 0, AUC = 0.9278338680667712
[15:37:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold = 1, AUC = 0.930528229950961
[15:37:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F

In [15]:
## entity_emebddings.py

import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

def create_model(data, catcols):
    """
    이 함수는 엔터티 임베딩을 위해 컴파일된 tf.keras 모델을 반환한다.
    :param data: pandas 데이터프레임
    :param catcols: 범주형 변수의 목록
    :return: 컴파일된 tf.keras 모델
    """
    # 임베딩을 위한 입력 목록
    inputs = []
    
    # 임베딩을 위한 출력 목록
    outputs = []
    
    # 모든 범주형 변수에 대해 계산
    for c in catcols:
        # 각 열의 유일 값의 수를 찾는다.
        num_unique_values = int(data[c].nunique())
        # 간단한 임베딩 사이즈 계산 방법
        #최소 값은 유일 값 수의 절반
        # 최대 값은 50. 최대 값 역시 유일 값의 수에 따라 다를 수 있지만
        # 50 이면 대부분의 경우 충분하다.
        # 유일 값의 개수가 수백 만이라면 더 높은 값이 필요할 것이다.
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
    
        # 사이즈 1 의 간단한 케라스 입력 레이어
        inp = layers.Input(shape=(1,))
    
        # 원래 입력에 임베딩 레이어를 추가한다.
        # 임베딩의 크기는 언제나 입력의 유일 값의 수 + 1 이다.
        out = layers.Embedding(
        num_unique_values + 1, embed_dim, name=c
        )(inp)

        # 1 차원 공간 드롭아웃은 임베딩 레이어에서 표준이다.
        # NLP 에서도 사용할 수 있다.
        out = layers.SpatialDropout1D(0.3)(out)
    
        # 입력의 차원을 임베딩에 맞게 변경한다.
        # 이 레이어가 현재 피쳐의 출력 레이어가 된다.
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
    
        # 입력을 입력 목록에 추가한다.
        inputs.append(inp)
        
        # 출력을 출력 목록에 추가한다.
        outputs.append(out)
        
    # 모든 출력을 연결한다.
    x = layers.Concatenate()(outputs)
    
    # 배치놈 레이어를 추가한다.
    # 여기서부터는 원하는데로 신경망 구조를 변경할 수 있다.
    # 아래는 내가 선호하는 구조이다.
    # 수치형 변수가 있다면 여기나 이전 concatenate()에 추가하면 된다.
    x = layers.BatchNormalization()(x)

    # 몇 개의 밀집 레이어를 드롭아웃과 함께 추가한다.
    # 1 개 혹은 2 개의 밀집 레이어로 시작하는 것이 좋다.
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    # 소프트멕스 활성 함수를 사용하여 두 개의 범주를 예측한다.
    # 이진 분류의 경우 시그모이드 활성함수를 사용하여 한 개의 값만
    # 출력할 수도 있다.
    y = layers.Dense(2, activation="softmax")(x)
    
    # 마지막 모델을 생성한다.
    model = Model(inputs=inputs, outputs=y)
    
    # 모델을 컴파일 한다.
    # 아담 최적화 함수와 이진 교차 엔트로피 손실 함수를 사용한다.
    # 다른 최적화 함수와 손실 함수를 시도해보고 모델 성능을 비교해 보라.
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model


def run(fold):
    # 폴드 값이 있는 학습 데이터를 불러 온다.
    df = pd.read_csv("cat_train_folds.csv")
    # id, target, kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
         f for f in df.columns if f not in ("id", "target", "kfold")
        ]
    
    # 모든 NaN 값을 NONE 으로 채운다.
    # 모든 열을 문자열로 변환함을 주목하라. 범주형으로 처리할 것이므로
    # 원래 타입이 무엇이든지 상관 없다
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")
    
    # 각각의 피쳐를 레이블인코딩한다.
    # 실무에서는 레이블인코더를 모두 저장해야 한다.
    for feat in features:
        lbl_enc = preprocessing.LabelEncoder()
        df.loc[:, feat] = lbl_enc.fit_transform(df[feat].values)

    # 폴드로 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # 폴드로 검증 데이터를 추출한다.
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    # tf.keras 모델을 생성한다.
    model = create_model(df, features)
    
    # 피쳐는 목록의 목록에 저장되어 있다.
    xtrain = [
        df_train[features].values[:, k] for k in range(len(features))
    ]
    
    xvalid = [
        df_valid[features].values[:, k] for k in range(len(features))
    ]
    
    # 타겟 변수를 추출한다.
    ytrain = df_train.target.values
    yvalid = df_valid.target.values
    
    # 타겟 변수를 이진화를 적용해 범주로 변환한다.
    ytrain_cat = utils.to_categorical(ytrain)
    yvalid_cat = utils.to_categorical(yvalid)

    # 모델을 학습한다.
    model.fit(xtrain,
            ytrain_cat,
            validation_data=(xvalid, yvalid_cat),
            verbose=1,
        batch_size=1024,
        epochs=3
        )

    # 검증 예측을 생성한다.
    valid_preds = model.predict(xvalid)[:, 1]
    # auc 값을 계산하여 출력한다.
    print(metrics.roc_auc_score(yvalid, valid_preds))
    # GPU 메모리를 반환한다.
    K.clear_session()
    
    
if __name__ == "__main__":
    run(0)
    run(1)
    run(2)
    run(3)
    run(4)

Epoch 1/3
469/469 [==============================] - 15s 25ms/step - loss: 0.5603 - val_loss: 0.4182
Epoch 2/3
469/469 [==============================] - 11s 24ms/step - loss: 0.4092 - val_loss: 0.3995
Epoch 3/3
469/469 [==============================] - 11s 24ms/step - loss: 0.4021 - val_loss: 0.3990
0.7840415903872121
Epoch 1/3
469/469 [==============================] - 15s 25ms/step - loss: 0.5572 - val_loss: 0.4162
Epoch 2/3
469/469 [==============================] - 11s 24ms/step - loss: 0.4094 - val_loss: 0.3990
Epoch 3/3
469/469 [==============================] - 12s 25ms/step - loss: 0.4022 - val_loss: 0.3990
0.7842122134771131
Epoch 1/3
469/469 [==============================] - 16s 27ms/step - loss: 0.5590 - val_loss: 0.4128
Epoch 2/3
469/469 [==============================] - 12s 26ms/step - loss: 0.4108 - val_loss: 0.3975
Epoch 3/3
469/469 [==============================] - 12s 25ms/step - loss: 0.4013 - val_loss: 0.3968
0.7872995942458795
Epoch 1/3
469/469 [===============